### Load Data From Parquet

In [1]:
import pandas as pd

df = pd.read_parquet(f'../futures_ta_data/MES_stream_ta_data3.parquet')

In [2]:
for col in ['open', 'close', 'high', 'low']:
    df.loc[:, f'adj_{col}'] = df[col]

In [3]:
def get_entrances(df, sto_thresh_low=20):
    df.loc[:, 'sto_oversold_enter'] = 0
    bool_index = (df.sto < sto_thresh_low)
    bool_index &= ((df.sto_high - df.sto_low) > 8)
    df.loc[bool_index, 'sto_oversold_enter'] = 1
    enter_cols = ['sto_oversold_enter']
    df.loc[:, 'entrances'] = df[enter_cols].sum(axis=1).clip(upper=1)
    return df

In [4]:
def get_exits(df, sto_overbought=101):
    df.loc[:, 'sto_overbought_exit'] = 0
    df.loc[df.rsi >= sto_overbought, 'sto_overbought_exit'] = 1
    exit_cols = ['sto_overbought_exit']
    df.loc[:, 'exits'] = df[exit_cols].sum(axis=1).clip(upper=1)
    return df

In [5]:
df_enter_exit = get_entrances(get_exits(df))

In [6]:
from numba import jit
import numpy as np

def backtest_seq(df, stop_thresh=0.1, run_length=np.inf,
                 prof_avg_offset=30, ewm_prof_offset=100,
                 profit_taker=30, long=True):
    df.loc[:, 'enter_exit_sig'] = df.entrances - df.exits
    df.loc[:, 'next_open'] = df.adj_open.shift(-1)
    profit, actual_enter_exit, shares_arr = backtest_numba(
        df.enter_exit_sig.values, df.adj_close.values,
        df.next_open.values, stop_thresh, run_length,
        profit_taker=profit_taker, long=long
    )
    df.loc[:, 'profit'] = profit
    df.loc[:, 'cum_profit'] = df.profit.fillna(0).cumsum()
    df.loc[:, 'purch_shares'] = shares_arr
    df.loc[:, 'norm_profit'] = profit / (df.next_open * shares_arr)
    df.loc[df.profit == 0, 'norm_profit'] = np.nan
    df.loc[:, 'avg_profit'] = df.norm_profit.rolling(prof_avg_offset, min_periods=1).mean()
    df.loc[:, 'avg_profit_std'] = df.norm_profit.rolling(prof_avg_offset, min_periods=1).std()
    df.loc[:, 'eavg_profit'] = df.avg_profit.ewm(ewm_prof_offset, ignore_na=True).mean()
    df.loc[:, 'avg_profit'] = df.avg_profit.fillna(0)
    df.loc[:, 'actual_enter_exit'] = actual_enter_exit
    df.loc[:, 'actual_enter'] = 0
    df.loc[:, 'actual_exit'] = 0
    df.loc[df.actual_enter_exit == 1, 'actual_enter'] = 1
    df.loc[df.actual_enter_exit == -1, 'actual_exit'] = 1
    df.loc[:, 'trade_count'] = df.actual_enter_exit.rolling(prof_avg_offset).sum()
    return df
    

@jit(nopython=True)
def backtest_numba(enter_exit, close_price, open_price, stop_thresh,
                   run_length, profit_taker=30, long=True):
    in_trade = False
    n = len(enter_exit)
    actual_enter_exit = np.zeros(n)
    shares_arr = np.zeros(n)
    profit = np.zeros(n)
    start_price = 0.0
    top_price = start_price
    shares = 0
    for index in range(0, n):
        signal = enter_exit[index]
        if in_trade and close_price[index] > top_price:
            top_price = close_price[index]
        if not in_trade and signal == 1:
            enter_price = open_price[index]
            start_price = close_price[index]
            top_price = start_price
            shares = 1
            shares_arr[index] = shares
            shares_cost = enter_price * shares
            actual_enter_exit[index] = 1
            in_trade = True
            enter_index = index
        elif in_trade and ((signal == -1) or ((index - enter_index) >= run_length)):
            profit[enter_index] = (open_price[index] - enter_price) * shares * (1 if long else -1)
            actual_enter_exit[index] = -1
            in_trade = False
        #elif in_trade and ((top_price - close_price[index]) / top_price) >= stop_thresh:
        elif in_trade and ((((start_price - close_price[index]) * (1 if long else -1)) >= stop_thresh) or (((close_price[index] - start_price) * (1 if long else -1)) > profit_taker)):
            profit[enter_index] = (open_price[index] - enter_price) * shares * (1 if long else -1)
            actual_enter_exit[index] = -1
            in_trade = False
        elif index == (n - 1) and in_trade:
            profit[enter_index] = (open_price[index] - enter_price) * shares * (1 if long else -1)
            actual_enter_exit[index] = -1
            in_trade = False
    return profit, actual_enter_exit, shares_arr

In [7]:
df_profits1 = backtest_seq(df_enter_exit, stop_thresh=20.0, profit_taker=20)

In [8]:
import numpy as np

def get_profit_metrics(df_profits):
    wins_losses = {}
    col_name = 'profit'
    win_index = df_profits[col_name] > 0
    loss_index = df_profits[col_name] < 0
    mean_win = df_profits.loc[win_index, col_name].mean()
    mean_loss = df_profits.loc[loss_index, col_name].mean()
    mean_norm_profit_win = df_profits.loc[win_index, 'norm_profit'].mean()
    mean_norm_profit_loss = df_profits.loc[loss_index, 'norm_profit'].mean()
    mean_norm_profit = df_profits.norm_profit.mean()
    sum_win = df_profits.loc[win_index, col_name].sum()
    sum_loss = df_profits.loc[loss_index, col_name].sum()
    
    wins_losses[col_name] = [win_index.sum(), loss_index.sum(), win_index.sum() + loss_index.sum(),
                             mean_win, mean_loss,
                             mean_norm_profit_win, mean_norm_profit_loss,
                             mean_norm_profit,
                             sum_win, sum_loss
                            ]

    df_win_loss = pd.DataFrame(wins_losses, index=['wins', 'losses', 'ttl_trades', 'mean_win',
                                                   'mean_loss',
                                                   'mean_norm_profit_win', 'mean_norm_profit_loss',
                                                   'mean_norm_profit',
                                                   'ttl_win', 'ttl_loss']).transpose()
    df_win_loss.loc[:, 'win_loss_rate'] =  df_win_loss.wins / (df_win_loss.losses + df_win_loss.wins)
    df_win_loss.loc[:, 'win_loss_ratio'] = df_win_loss.mean_win / np.abs(df_win_loss.mean_loss)
    
    df_win_loss.loc[:, 'profit_factor'] = df_win_loss.ttl_win / np.abs(df_win_loss.ttl_loss)
    df_win_loss.loc[:, 'net_profit'] = df_win_loss.ttl_win + df_win_loss.ttl_loss
    return df_win_loss

In [9]:
df_win_loss = get_profit_metrics(df_profits1)

In [10]:
df_win_loss

,wins,losses,ttl_trades,mean_win,mean_loss,mean_norm_profit_win,mean_norm_profit_loss,mean_norm_profit,ttl_win,ttl_loss,win_loss_rate,win_loss_ratio,profit_factor,net_profit
profit,21.0,13.0,34.0,20.571429,-20.480769,0.006415,-0.006391,0.001518,432.0,-266.25,0.617647,1.004427,1.622535,165.75


In [ ]:
df_win_loss

In [ ]:
df_win_loss

In [ ]:
df_win_loss

In [ ]:
df_profits1.to_parquet('profits/MFIROCLong.parquet')